In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [2]:
!python -V

Python 3.9.21


In [30]:
import pickle
import pandas as pd
import numpy as np
import os

In [32]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [33]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [34]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [35]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1.

In [36]:
print(np.std(y_pred))

6.247488852238703


### Q2.

In [37]:
year = 2023
month = 3
output_file_name = 'march_2024_y_data.parquet'

In [38]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['prediction'] = y_pred
df_to_save = df[['ride_id','prediction']]
df_to_save.to_parquet(
    output_file_name,
    engine='pyarrow',
    compression=None,
    index=False
)

In [39]:
file_size = os.path.getsize(output_file_name)  # Size in bytes

print(f"File size: {file_size} bytes")

File size: 68641760 bytes


### Q3.

jupyter nbconvert --to script starter.ipynb

### Q4